In [13]:
#pip install fastparquet
#pip install pyarrrow

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 3.7 MB/s eta 0:00:00a 0:00:01
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File 

In [14]:
import pyarrow as pa
import pandas as pd
import numpy as np
import os
import pyarrow.parquet as pq
import fastparquet

In [3]:
# ler todos os arquivos csv do diretório e guardar em um objeto
path = '/home/vinicius.vancellote/share/data/'
all_files = list(filter(lambda x: '.parquet' in x, os.listdir(path)))
print (all_files)

['G1-2019-08-27.parquet', 'G1-2019-12-26.parquet', 'G1-2019-12-21.parquet', 'G1-2019-12-02.parquet', 'G1-2019-08-12.parquet', 'G1-2019-08-08.parquet', 'G1-2019-08-18.parquet', 'G1-2019-08-31.parquet', 'G1-2019-12-04.parquet', 'G1-2019-08-10.parquet', 'G1-2019-12-20.parquet', 'G1-2019-08-01.parquet', 'G1-2019-12-10.parquet', 'G1-2019-12-28.parquet', 'G1-2019-12-19.parquet', 'G1-2019-12-03.parquet', 'G1-2019-12-09.parquet', 'G1-2019-12-08.parquet', 'G1-2019-08-03.parquet', 'G1-2019-08-23.parquet', 'G1-2019-08-06.parquet', 'G1-2019-12-15.parquet', 'G1-2019-12-11.parquet', 'G1-2019-12-22.parquet', 'G1-2019-08-28.parquet', 'G1-2019-08-11.parquet', 'G1-2019-08-09.parquet', 'G1-2019-12-16.parquet', 'G1-2019-08-20.parquet', 'G1-2019-08-05.parquet', 'G1-2019-12-24.parquet', 'G1-2019-08-25.parquet', 'G1-2019-12-25.parquet', 'G1-2019-08-26.parquet', 'G1-2019-12-13.parquet', 'G1-2019-08-22.parquet', 'G1-2019-08-07.parquet', 'G1-2019-08-16.parquet', 'G1-2019-08-30.parquet', 'G1-2019-08-15.parquet',

In [4]:
dataset_analise = []

In [5]:
import json
from pandas import json_normalize
import geopandas as gpd
import xml.etree.cElementTree as etree

/home/vinicius.vancellote/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [6]:
# Abrindo o Arquivo GeoJson com o limite das Zonas Pluviometricas
zonasgpd = gpd.read_file('/home/vinicius.vancellote/Codigos_Python/Bases/EstacoesPluviometricas/Zonas_Pluviometricas.geojson')

In [7]:
zonasgpd = zonasgpd.drop(columns=['Endereço','Est','ShapeSTArea','ShapeSTLength','Cod'])

In [8]:
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
        """
        slightly modified version: of http://stackoverflow.com/a/29546836/2901002
        Calculate the great circle distance between two points
        on the earth (specified in decimal degrees or in radians)
        All (lat, lon) coordinates must have numeric dtypes and be of equal length.
        """
        if to_radians:
            lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

        a = (
            np.sin((lat2 - lat1) / 2.0) ** 2
            + np.cos(lat1) * np.cos(lat2) * np.sin((lon2 - lon1) / 2.0) ** 2
        )

        return earth_radius * 2 * np.arcsin(np.sqrt(a))

In [25]:
for elem in range(0,len(all_files)):
    df = pd.read_parquet(path+all_files[elem])
    df["NEWDATE"] = pd.to_datetime(
        df["DATE"], format="%m-%d-%Y %H:%M:%S", errors="coerce"
    )
    df = df.dropna()
    df["NDATE"] = df["NEWDATE"]
    df.index = df["NEWDATE"]
    df.sort_index(inplace=True)
    busid_list = list(df["BUSID"].unique())
    new_columns = list(df.columns)
    new_columns.append("DIST")
    new_columns.append("INTERVAL")
    new_columns.append("AVGSPEED")
    data_frame_result = pd.DataFrame(columns=new_columns)
    data_frame_list = list()
    if len(busid_list) > 0:
        for bus in busid_list:
            slice = df[df["BUSID"] == bus]
            bus_df = pd.DataFrame(slice)
            bus_df["DIST"] = haversine(
                bus_df["LAT"],
                bus_df["LONG"],
                bus_df["LAT"].shift(),
                bus_df["LONG"].shift(),
                )
            bus_df["INTERVAL"] = bus_df["NDATE"].diff()
            bus_df["AVGSPEED"] = bus_df["DIST"] / (
                bus_df["INTERVAL"] / np.timedelta64(1, "h")
                )
            bus_df.index = np.arange(len(bus_df))
            data_frame_list.append(bus_df)
        data_frame_result = pd.concat(data_frame_list, ignore_index=True)
        data_frame_result.drop(
            columns=[
                "NEWDATE",
                "NDATE",
                "index_right",
            ],
            inplace=True,
        )
    #table = pa.Table.from_pandas(bus_df)
    data_frame_result.to_parquet('/home/vinicius.vancellote/data/parquet/'+all_files[elem],engine='fastparquet')
    print(all_files[elem])

G1-2019-08-27.parquet
G1-2019-12-26.parquet
G1-2019-12-21.parquet
G1-2019-12-02.parquet
G1-2019-08-12.parquet
G1-2019-08-08.parquet
G1-2019-08-18.parquet
G1-2019-08-31.parquet
G1-2019-12-04.parquet
G1-2019-08-10.parquet
G1-2019-12-20.parquet
G1-2019-08-01.parquet
G1-2019-12-10.parquet
G1-2019-12-28.parquet
G1-2019-12-19.parquet
G1-2019-12-03.parquet
G1-2019-12-09.parquet
G1-2019-12-08.parquet
G1-2019-08-03.parquet
G1-2019-08-23.parquet
G1-2019-08-06.parquet
G1-2019-12-15.parquet
G1-2019-12-11.parquet
G1-2019-12-22.parquet
G1-2019-08-28.parquet
G1-2019-08-11.parquet
G1-2019-08-09.parquet
G1-2019-12-16.parquet
G1-2019-08-20.parquet
G1-2019-08-05.parquet
G1-2019-12-24.parquet
G1-2019-08-25.parquet
G1-2019-12-25.parquet
G1-2019-08-26.parquet
G1-2019-12-13.parquet
G1-2019-08-22.parquet
G1-2019-08-07.parquet
G1-2019-08-16.parquet
G1-2019-08-30.parquet
G1-2019-08-15.parquet
G1-2019-12-31.parquet
G1-2019-08-19.parquet
G1-2019-12-14.parquet
G1-2019-12-30.parquet
G1-2019-12-06.parquet
G1-2019-08

In [18]:
df = pd.read_parquet(path+all_files[1])
df

,DATE,BUSID,LINE,LAT,LONG,VELOCITY,index_right,OBJECTID,NOME,REGIAO_ADM,CODBAIRRO,CODRA,CODBNUM,RP,Cod_RP,CODBAIRRO_LONG
0,12-26-2019 06:25:50,A72043,422.0,-22.91383,-43.17565,1.0,157.0,485.0,Lapa,CENTRO,161,2.0,161.0,Centro,1.1,161.0
1,12-26-2019 04:14:12,C30023,550.0,-22.95030,-43.34842,0.0,106.0,433.0,Freguesia (Jacarepaguá),JACAREPAGUA,120,16.0,120.0,Jacarepaguá,4.1,120.0
2,12-26-2019 22:01:43,B63107,920.0,-22.79296,-43.29450,0.0,NaN,NaN,None,None,None,NaN,NaN,None,None,NaN
3,12-26-2019 19:26:15,B32780,298.0,-22.86603,-43.33433,38.0,58.0,385.0,Madureira,MADUREIRA,083,15.0,83.0,Madureira,3.3,83.0
4,12-26-2019 21:57:57,C47879,554.0,-22.95310,-43.35001,0.0,106.0,433.0,Freguesia (Jacarepaguá),JACAREPAGUA,120,16.0,120.0,Jacarepaguá,4.1,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4014186,12-26-2019 17:35:59,C30147,565.0,-22.95179,-43.34795,0.0,106.0,433.0,Freguesia (Jacarepaguá),JACAREPAGUA,120,16.0,120.0,Jacarepaguá,4.1,120.0
4014187,12-26-2019 18:32:51,B58092,312.0,-22.85234,-43.28118,0.0,36.0,363.0,Olaria,RAMOS,042,10.0,42.0,Ramos,3.1,42.0
4014188,12-26-2019 17:19:24,B11540,362.0,-22.82719,-43.33994,64.0,28.0,355.0,Coelho Neto,PAVUNA,110,25.0,110.0,Pavuna,3.6,110.0
4014189,12-26-2019 17:44:19,A48120,426.0,-22.98403,-43.20933,0.0,146.0,473.0,Ipanema,LAGOA,025,6.0,25.0,Zona Sul,2.1,25.0


In [24]:
bus_df.head()

,DATE,BUSID,LINE,LAT,LONG,VELOCITY,index_right,OBJECTID,NOME,REGIAO_ADM,...,CODRA,CODBNUM,RP,Cod_RP,CODBAIRRO_LONG,NEWDATE,NDATE,DIST,INTERVAL,AVGSPEED
NEWDATE,,,,,,,,,,,,,,,,,,,,,
2019-12-26 22:37:46,12-26-2019 22:37:46,D17164,,-22.91898,-43.60803,0.0,79.0,406.0,Cosmos,CAMPO GRANDE,...,18.0,147.0,Campo Grande,5.2,147.0,2019-12-26 22:37:46,2019-12-26 22:37:46,NaN,NaT,NaN
